# Preliminaries

## Install and import libraries 

In [1]:
import torch
import torch.nn.functional as F
import os
import ast
import csv
import pandas as pd

## Set computation engine

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Connect to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load precomputed text features 

In [4]:
ProportionTrainVal=0.7

Xtrain_item_name = torch.load('/content/drive/MyDrive/data_rakuten/Xtrain_item_name.pt')
Xtrain_item_caption = torch.load('/content/drive/MyDrive/data_rakuten/Xtrain_item_caption.pt')
Ytrain_label = torch.load('/content/drive/MyDrive/data_rakuten/Ytrain_label.pt')
X_features=torch.cat((Xtrain_item_name,Xtrain_item_caption),0)

In [5]:
X_features.shape

torch.Size([1536, 212120])

In [6]:
class TextDataset(torch.utils.data.Dataset):

    def __init__(self,features,labels):
      #Load pre-computed tensors
      self.features=features
      self.labels=labels
        
    def __len__(self):
        return self.features.shape[1]

    def __getitem__(self, idx):

        return  self.features[:,idx],self.labels[:,idx]

trainSet= TextDataset(X_features[:,:int(X_features.shape[1]*ProportionTrainVal)],Ytrain_label[:,:int(X_features.shape[1]*ProportionTrainVal)])
trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=64,shuffle=True, num_workers=2)

valSet= TextDataset(X_features[:,int(X_features.shape[1]*ProportionTrainVal):],Ytrain_label[:,int(X_features.shape[1]*ProportionTrainVal):])
valLoader = torch.utils.data.DataLoader(valSet, batch_size=64,shuffle=False, num_workers=2)

In [7]:
len(valSet)+len(trainSet)==X_features.shape[1]

True

# Our prediction model : SVM approach

In [49]:
from sklearn import svm
import numpy as np
X = X_features.permute(1,0)[:10000,:] #(n_samples, n_features)
Y = Ytrain_label[1,:][:10000]  #n_samples
clf = svm.SVC()
clf.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [50]:
X_val = X_features.permute(1,0)[10000:12000,:] #(n_samples, n_features)
Y_val = Ytrain_label[1,:][10000:12000]  #n_samples
predictions=clf.predict(X_val)
accuracy=(np.array(predictions)==np.array(Y_val)).mean()
print(accuracy)

0.648


# Our prediction model : Neural approach

## Create the model

In [41]:
class CustomModel(torch.nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()

        #self.dropout = torch.nn.Dropout(0.1)
        self.fc1 = torch.nn.Linear(1536, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 19)


    def forward(self, text_features):
        text_features = F.relu(self.fc1(text_features))
        text_features = F.relu(self.fc2(text_features))
        #text_features = self.dropout(text_features)
        logits = self.fc3(text_features)

        return logits

model=CustomModel()
model.to(device)

CustomModel(
  (fc1): Linear(in_features=1536, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=19, bias=True)
)

## Train the model

In [42]:
nbr_labels_positive = torch.tensor([25673,71831,34014,33338,2383,8303,21697,28814,8353,12597,25017,10378,24582,10355,23583,12911,3325,51751,14534]) #number of labels
nbr_labels_negative = nbr_labels_positive.sum()*torch.ones(19)-nbr_labels_positive
coeffs = nbr_labels_negative/nbr_labels_positive    #coefficients for each label
coeffs = coeffs.to(device)

In [43]:
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=coeffs)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.15)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0

    for batch_idx, (inputs, targets) in enumerate(trainLoader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_idx%1000==0:
            print('{:.0f}%|Train Loss: {:.5f} '.format(100*batch_idx/(len(trainLoader)+1),train_loss/(batch_idx+1)))


In [ ]:
# training loop
for epoch in range(30):
    train(epoch)
    scheduler.step()

In [45]:
#Save weights
model_file = "/content/drive/MyDrive/data_rakuten/textmodel.pth"
torch.save(model.state_dict(), model_file)

In [46]:
#Load weights
model_file = "/content/drive/MyDrive/data_rakuten/textmodel.pth"
state_dict = torch.load(model_file)
model.load_state_dict(state_dict)

<All keys matched successfully>

# Generate csv file for submission

In [47]:
class TestDataset(torch.utils.data.Dataset):

    def __init__(self,):

        self.Xtest_item_name = torch.load('/content/drive/MyDrive/data_rakuten/Xtest_item_name.pt')
        self.Xtest_item_caption = torch.load('/content/drive/MyDrive/data_rakuten/Xtest_item_caption.pt')
        
        

    def __len__(self):
        return self.Xtest_item_name.shape[1]


    #all this processing needs to be done here because the output of __getitem__ needs to have a fixed size to use a BS>1
    def __getitem__(self, idx):
 
        return  torch.cat((self.Xtest_item_name[:,idx] ,self.Xtest_item_caption[:,idx]),0)

testSet= TestDataset()
testLoader = torch.utils.data.DataLoader(testSet, batch_size=1,shuffle=False, num_workers=2)

In [48]:
inv_dico_labels={ 0: "Beige",1:"Black",2:"Blue",3:"Brown",4:"Burgundy",5:"Gold",6:"Green",7:"Grey",
                 8:"Khaki",9:"Multiple Colors",10:"Navy",11:"Orange",12:"Pink",
                 13:"Purple",14:"Red",15:"Silver",16:"Transparent",17:"White",18:"Yellow"}

model.eval()

#Write prediction in the submission.csv file

with open('/content/drive/MyDrive/data_rakuten/submission.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow([',color_tags,'])
    with torch.no_grad():
        for batch_idx, inputs in enumerate(testLoader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            prediction=[]
            for indice,logits in enumerate(outputs.squeeze(0)):
                if logits>0: #put the tag if the proba is greater than 0.5
                    prediction.append(inv_dico_labels[indice]) 
            
            if len(prediction)>1:
                spamwriter.writerow(['{},"{}"'.format(batch_idx,prediction)])
            else:
                spamwriter.writerow(['{},{}'.format(batch_idx,prediction)])
            if batch_idx>300:
              break